In [12]:
!pip install fitz pandas openpyxl



In [10]:
!pip install PyMuPDF

In [15]:
!pip install pdfplumber pandas openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 61.2 MB/s eta 0:00:00


In [32]:
import pdfplumber
import pandas as pd
import re


def extract_tables_from_pdf(pdf_path, output_excel_path):
    with pdfplumber.open(pdf_path) as pdf:
        all_tables = []

        for i, page in enumerate(pdf.pages):
            try:
                tables = page.extract_tables()

                if tables:
                    for table in tables:
                        # Convert table data to a DataFrame
                        df = pd.DataFrame(table)
                        all_tables.append((f"Page_{i+1}_Table_{len(all_tables)+1}", df))
                else:
                    # Extract unstructured text for irregular tables
                    text = page.extract_text()
                    if text:
                        lines = text.split('\n')
                        data = [re.split(r'\s{2,}', line.strip()) for line in lines]
                        df = pd.DataFrame(data)
                        all_tables.append((f"Page_{i+1}_Unstructured_Table", df))
            except Exception as e:
                print(f"Error processing page {i+1}: {e}")

        if not all_tables:
            print(f"No tables found in {pdf_path}.")
            return

        # Save tables to Excel with each table on a different sheet
        with pd.ExcelWriter(output_excel_path) as writer:
            for name, df in all_tables:
                df.to_excel(writer, sheet_name=name, index=False)

    print(f"Tables extracted and saved to {output_excel_path}")


def decrypt_values(value):
    try:
        return ''.join([chr(int(value[i:i+2], 16)) for i in range(0, len(value), 2)])
    except:
        return value


def handle_encrypted_pdf(pdf_path, output_excel_path):
    with pdfplumber.open(pdf_path) as pdf:
        all_tables = []

        for i, page in enumerate(pdf.pages):
            tables = page.extract_tables()

            if tables:
                for table in tables:
                    decrypted_table = [[decrypt_values(cell) if isinstance(cell, str) else cell for cell in row] for row in table]
                    df = pd.DataFrame(decrypted_table)
                    all_tables.append((f"Page_{i+1}_Decrypted_Table", df))

        if not all_tables:
            print(f"No tables found in {pdf_path}.")
            return

        # Save tables to Excel with each table on a different sheet
        with pd.ExcelWriter(output_excel_path) as writer:
            for name, df in all_tables:
                df.to_excel(writer, sheet_name=name, index=False)

    print(f"Decrypted tables extracted and saved to {output_excel_path}")


if __name__ == "__main__":
    extract_tables_from_pdf("/content/test3 (1).pdf", "/content/test3 (1).xlsx")
    handle_encrypted_pdf("/content/test6 (1).pdf", "/content/test6 (1).xlsx")


Tables extracted and saved to /content/test3 (1).xlsx
No tables found in /content/test6 (1).pdf.
